<h3>Importing Libraries</h3>
<p>In this cell, the necessary Python libraries such as pandas, yfinance, and datetime are imported for data handling and time series analysis.</p>

In [ ]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta

<h3>Data Collection</h3>
<p>This cell retrieves historical stock market data for Google (GOOG) from Yahoo Finance API. It defines the date range for data collection and selects relevant columns.</p>

In [ ]:
today = date.today()
end_date = today.strftime("%Y-%m-%d")

d1 = date.today() - timedelta(days=365)
start_date = d1.strftime("%Y-%m-%d")

data = yf.download('GOOG', 
                      start=start_date, 
                      end=end_date, 
                      progress=False)
data["Date"] = data.index
df = data[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]]
df.reset_index(drop=True, inplace=True)
print(df.tail())

<h3>Data Preprocessing</h3>
<p>Here, the data is further preprocessed. It narrows down the dataset to include only the 'Date' and 'Close' columns. The code also resets the index for data consistency.</p>

In [ ]:
df = df[["Date", "Close"]]
print(df.head())

<h3>Data Visualization</h3>
<p>This cell uses the matplotlib library to create a time series plot of the closing prices of Google's stock. The 'fivethirtyeight' style is applied for visualization.</p>

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.figure(figsize=(15, 10))
plt.plot(df["Date"], df["Close"])

<h3>Time Series Decomposition</h3>
<p>The statsmodels library is utilized for seasonal decomposition. This cell separates the trend, seasonality, and residual components of the stock prices.</p>

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df["Close"], 
                            model='multiplicative', period = 30)
fig = plt.figure()  
fig = result.plot()  
fig.set_size_inches(15, 10)

<h3>AutoCorrelation Plot</h3>
<p>AutoCorrelation is visualized using the pandas library, helping identify potential time series patterns.</p>

In [ ]:
pd.plotting.autocorrelation_plot(df["Close"])

<h3>Partial AutoCorrelation Plot</h3>
<p>This cell generates the Partial AutoCorrelation Function (PACF) plot to aid in parameter selection for the ARIMA model.</p>

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(df["Close"], lags = 100)

<h3>ARIMA Modeling</h3>
<p>The ARIMA (AutoRegressive Integrated Moving Average) model is constructed here. It fits the model to the 'Close' prices with specified parameters and prints a summary of the model.</p>

In [ ]:
p, d, q = 5, 1, 2
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(df["Close"], order=(p,d,q))  
fitted = model.fit()  
print(fitted.summary())

<h3>ARIMA Predictions</h3>
<p>This cell generates predictions using the ARIMA model and prints the results.</p>

In [ ]:
predictions = fitted.predict()
print(predictions)

<h3>SARIMAX Predictions</h3>
<p>Predictions for future stock prices are generated using the SARIMAX model.</p>

In [ ]:
import statsmodels.api as sm
model=sm.tsa.statespace.SARIMAX(data['Close'],
                                order=(p, d, q),
                                seasonal_order=(p, d, q, 12))
model=model.fit()
print(model.summary())

<h3>Data and Predictions Visualization</h3>
<p>The cell plots the training data and the predictions on a single graph for visual comparison.</p>

In [ ]:
# Generate predictions using the SARIMAX model

start_date = (df["Date"].iloc[-1] + pd.Timedelta(days=1)).strftime("%Y-%m-%d")
date_range = pd.date_range(start=start_date, periods=100, freq="D")
predictions = model.predict(len(df), len(df) + 99)
                            
predictions.index = date_range
print(predictions)

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(df["Date"], df["Close"])
plt.plot(predictions)